In [1]:
from pyspark.sql import SparkSession

Inciando a sessão spark, que pode ser vista como uma manifestação do driver process para o usuário 

In [2]:
spark = SparkSession.builder.getOrCreate()


Definindo a leitura do arquivo. 
A leitura de um arquivo, não e uma action dentro do spark, por conta disso, ela será apenas inserida a uma pilha de transformações, que serão todas executadas no momento em que uma action for invocada

Interessante saber que, nesse momento meu df, possui um determinado numero de colunas, porém um indeterminado numero de linhas, isso se dá devido a ação de leitura ser executada em lazy evaluation

In [5]:
df_bitcoin = spark.read.option('inferSchema', 'true'
                        ).option('header', 'true'
                        ).csv('./data/archive/btcusd_1-min_data.csv')
df_bitcoin

DataFrame[Timestamp: double, Open: double, High: double, Low: double, Close: double, Volume: double]

Utilizando a action take

In [6]:
df_bitcoin.take(2)

[Row(Timestamp=1325412060.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412120.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0)]

In [16]:
df_bitcoin.show()

+------------+----+----+----+-----+------+
|   Timestamp|Open|High| Low|Close|Volume|
+------------+----+----+----+-----+------+
|1.32541206E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541212E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541218E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541224E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254123E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541236E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541242E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541248E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541254E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254126E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541266E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541272E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541278E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541284E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254129E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541296E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541302E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541308E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541314E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254132E9|4.58|4.58|4.58| 4.58|   0.0|
+----------

### Construindo nossa primeira transformação

Aqui, vou realizar uma transformação do tipo sort. 
Por definição a ordenação de uma linha em relação a outras não pode ser tratada como uma narrow transformation.
Uma vez que a ordenação depende das outras linhas, por conta idsso trata-se de uma wide transformations. (quando o processamento é feito em disco)

Utilizando o explain

In [18]:
df_bitcoin.sort('Timestamp').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Timestamp#42 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(Timestamp#42 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=85]
      +- FileScan csv [Timestamp#42,Open#43,High#44,Low#45,Close#46,Volume#47] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/a:/DS/Spark/learningSpark/data/archive/btcusd_1-min_data.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Timestamp:double,Open:double,High:double,Low:double,Close:double,Volume:double>




Alternado a configuração do spark, para retornar 5 arquivos de particao como saida 

In [19]:
spark.conf.set('spark.sql.shuffle.partitions', '5')


In [20]:
df_bitcoin.sort('Timestamp').take(8)

[Row(Timestamp=None, Open=57854.0, High=57864.0, Low=57835.0, Close=57835.0, Volume=1.35346619),
 Row(Timestamp=1325412060.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412120.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412180.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412240.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412300.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412360.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0),
 Row(Timestamp=1325412420.0, Open=4.58, High=4.58, Low=4.58, Close=4.58, Volume=0.0)]